# TASK 2 - Adding data to SQL - If we have only monthky snapshits to add (incremental load/delta) then we iterate over the rows in excel.
I am using data frames here too, but other objects like dict can be used to read rows from excel. The idea is to loop over one by one and increment duration in ticker_info table while adding data to nifty50_monthly.

In [ ]:
import pymysql
import pandas as pd
import numpy as np
from google.cloud.sql.connector import Connector
import sqlalchemy
import datetime
# from pymysql.err import IntegrityError, OperationalError
from sqlalchemy.exc import OperationalError,ProgrammingError,IntegrityError

# The Cloud SQL Python Connector can be used along with SQLAlchemy using the
# 'creator' argument to 'create_engine'
connector = Connector()
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        "my-project-olam-95:us-central1:my-case-study-1",
        "pymysql",
        user="root",
        password="uIF2eu1AKgIlqHNv",
        db="nifty-data",
    )
    return conn

pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

db = pool.connect()

db.execute("DROP TABLE nifty50_monthly")
db.execute("DROP TABLE ticker_info")

create_ticker_info = "CREATE TABLE ticker_info ( symbol VARCHAR(20) PRIMARY KEY, name VARCHAR(50), industry VARCHAR(40), duration_in_nifty50 INT);"

create_nifty50_monthly = "CREATE TABLE nifty50_monthly ( symbol VARCHAR(20),report_date DATE,equity_capital BIGINT,free_float_market_cap DECIMAL,weightage DECIMAL,beta DECIMAL,R2 DECIMAL,volatility_per DECIMAL,monthly_return DECIMAL,avg_impact_cost_percent DECIMAL,PRIMARY KEY (report_date, symbol),FOREIGN KEY (symbol) REFERENCES ticker_info(symbol));"

try:
    db.execute(create_ticker_info)
    db.execute(create_nifty50_monthly)
except (OperationalError, ProgrammingError) as err:
    print(err.args[0])
else:
    print("OK")
    

insert_stmt_ticker_info = sqlalchemy.text("INSERT INTO ticker_info (symbol, name, industry, duration_in_nifty50) VALUES (:symbol, :name, :industry, :duration) ON DUPLICATE KEY UPDATE duration_in_nifty50=duration_in_nifty50+1")

insert_stmt_nifty50_monthly = sqlalchemy.text("INSERT INTO nifty50_monthly (symbol, report_date, equity_capital, free_float_market_cap, weightage, beta, R2, volatility_per, monthly_return, avg_impact_cost_percent) VALUES (:symbol, :report_date, :equity_capital, :free_float_market_cap, :weightage, :beta, :R2, :volatility_per, :monthly_return, :avg_impact_cost_percent)")

for num, row in df.iterrows():
    try:
        db.execute(insert_stmt_ticker_info,symbol=row[0],name=row[1],industry=row[2],duration=1)
    except (OperationalError, ProgrammingError) as err:
        print("FAILED to Add/Update table ticker_info",err)
    else:
        try:
            db.execute(insert_stmt_nifty50_monthly,symbol=row[0],report_date=row[11],equity_capital=row[3],free_float_market_cap=row[4],weightage=row[5],beta=row[6],R2=row[7],volatility_per=row[8],monthly_return=row[9],avg_impact_cost_percent=row[10])
        except (OperationalError, ProgrammingError) as err:
            print("FAILED to Add/Update table insert_stmt_nifty50_monthly",err)
        else:
            print("Inserted into both tables.")
        
    